# Greenplum Database  Concepts Explained - Part 1
## 1. System Setup
### 1.1 Initialize database connection and setup global variable values

In [ ]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

%reload_ext sql
%sql $CONNECTION_STRING

In [ ]:
%%sql $DB_USER@$DB_SERVER
SHOW gp_autostats_mode;
ALTER DATABASE gpadmin SET gp_autostats_mode TO 'NONE';
SHOW gp_autostats_mode;

In [ ]:
%%sql $DB_USER@$DB_SERVER
SELECT version();

## 2. The Amazon Customer Reviews Dataset

Over 130+ million customer reviews are available to researchers as part of this release. The data is available in TSV files in the `amazon-reviews-pds` S3 bucket in AWS US East Region. Each line in the data files corresponds to an individual review (tab delimited, with no quote and escape characters). Samples of the data are available in English and French; more details on the information in each column can be found [here](https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt).

If you use the AWS Command Line Interface, you can list data in the bucket with the `aws s3 ls` command:

`aws s3 ls s3://amazon-reviews-pds/tsv/`

To download data using the AWS Command Line Interface, you can use the `aws s3 cp` command. For instance, the following command will copy the file named `amazon_reviews_us_Camera_v1_00.tsv.gz` to your local directory:

`aws s3 cp s3://amazon-reviews-pds/tsv/<S3 File> <Local File>`

### 2.1 Prepare AWS System and setup awscli library via pip

In [ ]:
shfilecode = !pygmentize -f html -O full,style=friendly -l shell script/1-1-system-prepare.sh
display_html('\n'.join(shfilecode), raw=True)

In [ ]:
!ssh-keygen -R $DB_SERVER
!ssh-keyscan $DB_SERVER >> ~/.ssh/known_hosts
!scp -i ~/.ssh/aws-gp.pem script/1-1-system-prepare.sh $DB_USER@$DB_SERVER:system-prepare.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./system-prepare.sh'
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'sudo ./system-prepare.sh'

### 2.2 Provide AWS Access Key ID & Secret Access Key

In [ ]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/1-2-aws-configure.sh
display_html('\n'.join(shfilecode), raw=True)

In [ ]:
import getpass

!scp -i ~/.ssh/aws-gp.pem script/1-2-aws-configure.sh $DB_USER@$DB_SERVER:aws-configure.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./aws-configure.sh'

cmd = 'sudo ./aws-configure.sh ' 
cmd = cmd + getpass.getpass("AWS Access Key ID [None]:") 
cmd = cmd + ' ' + getpass.getpass("AWS Secret Access Key [None]:")

!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

### 2.3 Copy source files from AWS S3
For our demo, we choose to download the available files into the `/home/gpadmin/data/` folder, using the `aws s3 cp <S3 File> <Local File>` command described before, as follows:

In [ ]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/1-3-aws-s3-copy.sh
display_html('\n'.join(shfilecode), raw=True)

In [ ]:
!scp -i ~/.ssh/aws-gp.pem script/1-3-aws-s3-copy.sh $DB_USER@$DB_SERVER:aws-s3-copy.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./aws-s3-copy.sh'
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'sudo ./aws-s3-copy.sh'

## 3. Data Loading
### 3.1. Create the Schema (optional) and the Database Table to hold the dataset, as shown below:

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/2-1-create-db-schema-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

In [ ]:
query = !cat script/2-1-create-db-schema-table.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/2-2-count-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

In [ ]:
query = !cat script/2-2-count-table.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

### 3.2. Load the Input Dataset using the gpload Utility
**gpload** is a data loading utility that acts as an interface to the Greenplum Database external table parallel loading feature. Using a load specification defined in a YAML formatted control file, gpload executes a load by invoking the Greenplum Database parallel file server (**gpfdist**), creating an external table definition based on the source data defined, and executing an *INSERT*, *UPDATE* or *MERGE* operation to load the source data into the target table in the database.

You can declare more than one file as input/source as long as the data is of the same format in all files specified. Additionally, if the files are compressed using **gzip** or **bzip2** (have a .gz or .bz2 file extension), the files will be uncompressed automatically (provided that gunzip or bunzip2 is in your path). You can also declare options such as the schema of the source data files, perform basic transformations, define custom delimiter and/or escape character(s), and many more. For the full list of available options, check the GPLoad Utility Reference available on [Pivotal Greenplum Database Documentation](https://gpdb.docs.pivotal.io/latest) (*Pivotal Greenplum Documentation > Utility Guide > Management Utility Reference > gpload*).

The operation, including any SQL commands specified in the SQL collection of the YAML control file, are performed as a single transaction to prevent inconsistent data when performing multiple, simultaneous load operations on a target table.

For our demo, we have prepared the *gpload_amzn_reviews.yaml* YAML control file, as shown here:

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l yaml script/3-2-gpload-amzn-reviews.yaml
display_html('\n'.join(sqlfilecode), raw=True)

#### 3.2.1. Delete error log information for existing tables in the current database.

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/3-1-delete-error-log-info.sql
display_html('\n'.join(sqlfilecode), raw=True)

In [ ]:
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'if [ -f ./gpload_amzn_reviews.log ]; then rm ./gpload_amzn_reviews.log; fi'

query = !cat script/3-1-delete-error-log-info.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

#### 3.2.2. Copy GPLoad YAML file across to the Database Server and execute

In [ ]:
!scp -i ~/.ssh/aws-gp.pem script/3-2-gpload-amzn-reviews.yaml $DB_USER@$DB_SERVER:gpload_amzn_reviews.yaml

cmd = "gpload -d {0} -f ./gpload_amzn_reviews.yaml -l ./gpload_amzn_reviews.log 2>&1".format(DB_USER) 
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

### 3.3. Check gpload execution

Check **gpload** execution output (shown above and also available on *./gpload_amzn_reviews.log*), confirm successful loading of the data and/or identify any message which require ones attention and/or actions:

#### 3.3.1. Check the data has been properly loaded, by confirming row count shown above:

In [ ]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/3-3-count-amzn-reviews.sql
display_html('\n'.join(sqlfilecode), raw=True)

In [ ]:
query = !cat script/3-3-count-amzn-reviews.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

In [ ]:
cmd = 'cat /home/gpadmin/gpload_amzn_reviews.log\
    | grep -e '"'"'WARN|select'"'"'\
    | awk '"'"'BEGIN{FS="|";OFS=" "} {print $3}'"'"'\
    | awk '"'"'{print $1, "COUNT(*)", $3, $4, $5, $6, $7, $8}'"'"''
query = !ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd
%sql $DB_USER@$DB_SERVER {''.join(query)}

#### 3.3.3. Check a sample set of 10 rows from the data formatting errors, if such were identified by the gpload execution log:

In [ ]:
cmd = 'cat /home/gpadmin/gpload_amzn_reviews.log\
    | grep -e '"'"'WARN|select'"'"'\
    | awk '"'"'BEGIN{FS="|"} {print $3, "LIMIT 10"}'"'"' ' 
query = !ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd
%sql $DB_USER@$DB_SERVER {''.join(query)}

### Continue to Part 2 of Greenplum Database Concepts Explained; [Basic Table Functions](AWS-GP-demo-1.ipynb).